# 📌 Adım 3 – `cv::Mat → torch::Tensor` Dönüşümü (OpenCV → LibTorch)

Bu adımda amacım, OpenCV ile okuduğum bir görüntüyü (`cv::Mat`) LibTorch’un anlayacağı forma (`torch::Tensor`) dönüştürmek.

Genel hedef:

1. OpenCV ile resmi okumak (`cv::imread`)
2. BGR → RGB dönüşümü
3. `uint8` → `float32` ve 0–1 aralığına ölçekleme
4. `HWC` (height, width, channels) → `NCHW` (batch, channels, height, width) dönüşümü
5. Ortaya çıkan tensörün şeklini kontrol etmek

Bu yapı, daha sonra PyTorch’ta eğittiğim CNN modellerini C++ tarafında beslemek için temel şablon olacak.

---

## 🔹 1) OpenCV resmi BGR formatında okur

OpenCV ile resim okuduğumda:

```cpp
cv::Mat img = cv::imread("test.jpg");
```

* Tür genelde: CV_8UC3 (8-bit, 3 kanal, BGR)

* Kanal sırası: BGR

PyTorch modellerinde ise genelde:

* Kanal sırası: RGB

* Tensor formatı: N x C x H x W (batch, channels, height, width)

* Bu yüzden hem kanal sırasını hem de boyut sırasını dönüştürmem gerekiyor.

---
## 🔹 2) Adım adım dönüşüm mantığı

* BGR → RGB

* uint8 (0–255) → float32 + 0–1 aralığı

* H x W x C → 1 x C x H x W (batch boyutu 1 olacak)

* Tensörü torch::from_blob ile LibTorch içine taşımak

---
## 🔹 3) Örnek C++ Kodu – cv::Mat → torch::Tensor

Aşağıdaki kod, daha önce çalıştırdığım OpenCV testine devam niteliğinde:

```cpp

// SORU:
//  - OpenCV ile okuduğum cv::Mat görüntüsünü, LibTorch için uygun
//    bir torch::Tensor haline nasıl getiririm?

#include <iostream>
#include <torch/script.h>
#include <opencv2/opencv.hpp>

int main() {
    try {
        // 1) Resmi OpenCV ile oku
        cv::Mat img = cv::imread("test.jpg");
        if (img.empty()) {
            std::cerr << "[HATA] test.jpg yuklenemedi..." << std::endl;
            return -1;
        }

        std::cout << "[OK] Orijinal resim boyutu: "
                  << img.cols << "x" << img.rows << " (W x H)" << std::endl;

        // 2) BGR -> RGB dönüşümü (PyTorch genelde RGB bekler)
        cv::cvtColor(img, img, cv::COLOR_BGR2RGB);

        // 3) uint8 -> float32 ve 0–1 aralığına ölçekleme
        cv::Mat img_float;
        img.convertTo(img_float, CV_32F, 1.0 / 255.0);

        // 4) from_blob ile HWC -> tensor (N, H, W, C)
        //    Not: img_float.rows = H, img_float.cols = W, 3 kanal var.
        auto tensor_image = torch::from_blob(
            img_float.data,                                        // veri pointer'ı
            {1, img_float.rows, img_float.cols, 3},                // boyutlar: N, H, W, C
            torch::TensorOptions().dtype(torch::kFloat32)          // tip: float32
        );

        // 5) NHWC -> NCHW permute (PyTorch standart formatı)
        tensor_image = tensor_image.permute({0, 3, 1, 2}).contiguous();

        std::cout << "[OK] Tensor boyutu (NCHW): " << tensor_image.sizes() << std::endl;

        // *** Önemli Not:
        // from_blob, OpenCV'nin bellek alanını kullanıyor.
        // img/img_float yaşam döngüsü sonlanmadan bu tensörü kullanmam gerekiyor.
        // Gerekirse .clone() ile kendi kopyasını oluşturabilirim:
        //
        // tensor_image = tensor_image.clone();

        // Bu aşamada tensor_image artık modele verilebilecek durumda:
        // - Şekil: [1, 3, H, W]
        // - Tip: float32
        // - Değer aralığı: 0–1
    }
    catch (const std::exception& e) {
        std::cerr << "[EXCEPTION] " << e.what() << std::endl;
        return -1;
    }

    return 0;
}
```


---
## 🔹 4) from_blob ve .clone() İnceliği

* torch::from_blob, veriyi kopyalamaz, sadece mevcut bellek üzerinde bir view oluşturur.

* Yani img_float serbest bırakılırsa, tensör de bozulur.

Eğer tensörü daha sonra, img’den bağımsız kullanmak istiyorsam:

>tensor_image = tensor_image.clone();


* yaparak LibTorch'un kendi belleğine kopyalamam gerekir.

-----

## CMakeLists dosyası için;

```bash
cmake_minimum_required(VERSION 3.20 FATAL_ERROR)
project(libtorch_opencv_test)

# LibTorch yolu
set(CMAKE_PREFIX_PATH "C:/libtorch")

# OpenCV CMake config klasörü (sende burası)
set(OpenCV_DIR "C:/opencv/build/x64/vc16/lib")

# OpenCV ve Torch'u bul
find_package(OpenCV REQUIRED)
find_package(Torch REQUIRED)

# Derlenecek dosya
add_executable(model_test main.cpp)

# OpenCV include klasörleri
include_directories(${OpenCV_INCLUDE_DIRS})

# Kütüphaneleri bağla
target_link_libraries(model_test 
    "${TORCH_LIBRARIES}"
    ${OpenCV_LIBS}
)

# C++ standardı
set_property(TARGET model_test PROPERTY CXX_STANDARD 17)

# LibTorch DLL'lerini exe klasörüne kopyala
add_custom_command(TARGET model_test POST_BUILD
    COMMAND ${CMAKE_COMMAND} -E copy_directory
    "C:/libtorch/bin"
    $<TARGET_FILE_DIR:model_test>
)
```



---

## Bu işlemler sonucunda çıktı şu şekilde olacak 

```bash
[OK] Orijinal resim boyutu: 282x179 (W x H)
[OK] Tensor boyutu (NCHW): [1, 3, 179, 282]


----